In [87]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


In [55]:
df= pd.read_excel('Online-Retail.xlsx')

In [56]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [57]:

# Check for missing values
if df.isnull().values.any():
    print("The dataframe contains missing values")
else:
    print("The dataframe does not contain missing values")

# Check for duplicate values
if df.duplicated().values.any():
    print("The dataframe contains duplicate values")
else:
    print("The dataframe does not contain duplicate values")

The dataframe contains missing values
The dataframe contains duplicate values


In [58]:
df.dropna(inplace=True)

# remove duplicates
df.drop_duplicates(inplace=True)

In [59]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [82]:
df.info()
df.describe()
print(df[['Quantity', 'UnitPrice', 'TotalAmount']].describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 401603
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     401604 non-null  object        
 1   StockCode     401604 non-null  object        
 2   Description   401604 non-null  object        
 3   Quantity      401604 non-null  int64         
 4   InvoiceDate   401604 non-null  datetime64[ns]
 5   UnitPrice     401604 non-null  float64       
 6   CustomerID    401604 non-null  float64       
 7   Country       401604 non-null  object        
 8   TotalAmount   401604 non-null  float64       
 9   InvoiceMonth  401604 non-null  int64         
 10  InvoiceDay    401604 non-null  int64         
 11  Recency       401604 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(4), object(4)
memory usage: 39.8+ MB
            Quantity      UnitPrice    TotalAmount
count  401604.000000  401604.000000  40160

In [61]:
# Check for missing values
if df.isnull().values.any():
    print("The dataframe contains missing values")
else:
    print("The dataframe does not contain missing values")

# Check for duplicate values
if df.duplicated().values.any():
    print("The dataframe contains duplicate values")
else:
    print("The dataframe does not contain duplicate values")

The dataframe does not contain missing values
The dataframe does not contain duplicate values


In [62]:


# Convert the InvoiceDate column to datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Create a TotalAmount column by multiplying Quantity and UnitPrice
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

# Create an InvoiceMonth column by extracting the month from the InvoiceDate column
df['InvoiceMonth'] = df['InvoiceDate'].dt.month

# Create an InvoiceDay column by extracting the day of the week from the InvoiceDate column
df['InvoiceDay'] = df['InvoiceDate'].dt.dayofweek

# Group the data by CustomerID and calculate the most recent date for each customer
most_recent_date = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
most_recent_date.columns = ['CustomerID', 'MostRecentDate']

# Calculate the Recency feature by subtracting the most recent date from the current date
most_recent_date['Recency'] = (pd.to_datetime('2023-03-13') - most_recent_date['MostRecentDate']).dt.days

lambda x: (pd.to_datetime('2023-03-13') - x.max())

# Merge the Recency feature back into the original DataFrame
df = pd.merge(df, most_recent_date[['CustomerID', 'Recency']], on='CustomerID')

df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount,InvoiceMonth,InvoiceDay,Recency
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,12,2,4413
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2,4413
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,12,2,4413
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2,4413
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2,4413


In [92]:
# Group the data by CustomerID and calculate the summary statistics
customer_data = df.groupby('CustomerID').agg({
    'TotalAmount': 'sum',
    'StockCode': 'nunique',
    'InvoiceNo': 'nunique',
    'InvoiceDate': lambda x: (pd.to_datetime('2023-03-13') - x.max()).days,
    'InvoiceNo': 'count'
})

# Calculate the AvgTransactionAmount feature
customer_data['AvgTransactionAmount'] = customer_data['TotalAmount'] / customer_data['InvoiceNo']

# Rename the columns
customer_data.columns = ['TotalAmount', 'NumUniqueProducts', 'NumTransactions', 'Recency', 'AvgTransactionAmount']



# Scale the features using StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(customer_data)

# Convert the scaled data back to a DataFrame
scaled_data_df = pd.DataFrame(scaled_data, columns=customer_data.columns, index=customer_data.index)

customer_data.head(100) 


,TotalAmount,NumUniqueProducts,NumTransactions,Recency,AvgTransactionAmount
CustomerID,,,,,
12346.0,0.00,1,2,4436,0.000000
12347.0,4310.00,103,182,4113,23.681319
12348.0,1797.24,22,31,4186,57.975484
12349.0,1757.55,73,73,4129,24.076027
12350.0,334.40,17,17,4421,19.670588
...,...,...,...,...,...
12462.0,1124.01,51,72,4113,15.611250
12463.0,1303.63,66,88,4157,14.813977
12464.0,1212.05,6,29,4121,41.794828


In [93]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(scaled_data)

# Add the cluster labels to the DataFrame
customer_data['Cluster'] = kmeans.labels_

# Print the number of customers in each cluster
print(customer_data['Cluster'].value_counts())

0    3201
1    1157
2      14
Name: Cluster, dtype: int64


In [94]:
# Filter the DataFrame to include only cluster 0
cluster_0 = customer_data[customer_data['Cluster'] == 0]

# Calculate the mean of the features for cluster 0
mean_features_0 = cluster_0.mean()

# Print the mean features for cluster 0
print(mean_features_0)

TotalAmount             1955.993569
NumUniqueProducts         71.610434
NumTransactions          104.197751
Recency                 4149.130584
AvgTransactionAmount      28.914767
Cluster                    0.000000
dtype: float64
